# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [1]:
! git clone https://github.com/Corentin312/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch

Cloning into 'exam_2025'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 21), reused 20 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 1.41 MiB | 16.56 MiB/s, done.
Resolving deltas: 100% (21/21), done.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [2]:
mySeed = 312

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [3]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

In [5]:
# On peut utiliser de la regression linéaire

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 1. Préparer les données
X_train = train_set['inputs']  # Matrice des prédicteurs (x, y, z) pour l'entraînement
y_train = train_set['targets']  # Vecteur des cibles (t) pour l'entraînement
X_test = test_set['inputs']    # Matrice des prédicteurs pour le test
y_test = test_set['targets']    # Vecteur des cibles pour le test

# 2. Créer un modèle de régression linéaire
model = LinearRegression()

# 3. Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

# 4. Afficher les coefficients estimés (theta_k)
print("theta0 :", model.intercept_)
print("theta 1 a 3 :", model.coef_)

# 5. Prédire les valeurs pour les données de test
predictions = model.predict(X_test)

# 6. Calculer l'erreur quadratique moyenne (MSE)
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error test :", mse)

theta0 : 15.576677853661185
theta 1 a 3 : [3.04507485 3.09796146 6.2816341 ]
Mean Squared Error test : 4.1952892768798575


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

In [7]:
# Un simple reseau lineaire suffit pour ce cas la car on cherche a minimiser t- θ0 + θ1x + θ2y + θ3z avec une MSE

import torch.nn as nn

# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear = nn.Linear(3, 1)  # 3 entrées (x, y, z), 1 sortie (t), la generalisation se fait en changeant le nb 3

    def forward(self, x):
        return self.linear(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [8]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()

        outputs = mySimpleNet(batch_inputs.float())

        loss = criterion(outputs, batch_targets.float().unsqueeze(1))

        loss.backward()

        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/500], Loss: 239.5751
Epoch [2/500], Loss: 158.1189
Epoch [3/500], Loss: 96.4908
Epoch [4/500], Loss: 52.6864
Epoch [5/500], Loss: 31.8644
Epoch [6/500], Loss: 26.9871
Epoch [7/500], Loss: 14.8501
Epoch [8/500], Loss: 10.5724
Epoch [9/500], Loss: 6.3731
Epoch [10/500], Loss: 6.8010
Epoch [11/500], Loss: 6.1861
Epoch [12/500], Loss: 5.9058
Epoch [13/500], Loss: 5.6784
Epoch [14/500], Loss: 5.2387
Epoch [15/500], Loss: 5.7223
Epoch [16/500], Loss: 3.9443
Epoch [17/500], Loss: 4.3086
Epoch [18/500], Loss: 5.0385
Epoch [19/500], Loss: 4.5024
Epoch [20/500], Loss: 4.5153
Epoch [21/500], Loss: 4.1080
Epoch [22/500], Loss: 4.2589
Epoch [23/500], Loss: 4.3054
Epoch [24/500], Loss: 4.1815
Epoch [25/500], Loss: 4.6702
Epoch [26/500], Loss: 3.7221
Epoch [27/500], Loss: 4.8442
Epoch [28/500], Loss: 4.5919
Epoch [29/500], Loss: 3.8462
Epoch [30/500], Loss: 3.7651
Epoch [31/500], Loss: 4.0006
Epoch [32/500], Loss: 3.9025
Epoch [33/500], Loss: 4.1443
Epoch [34/500], Loss: 3.6227
Epoch [35/500

**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

In [11]:
print("theta0 :", mySimpleNet.linear.bias.data)
print("theta 1 a 3 :", mySimpleNet.linear.weight.data)
# On retrouve bien des pods similaires

theta0 : tensor([15.5762])
theta 1 a 3 : tensor([[3.0447, 3.0989, 6.2816]])


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [13]:
X_test_tensor = torch.tensor(test_set['inputs']).float()
predictions_nn = mySimpleNet(X_test_tensor).detach().numpy()

mse_nn = mean_squared_error(test_set['targets'], predictions_nn)

print(mse_nn)

# On trouve les memes resultats, cela s'explique par le fait qu'un reseau de neurone a une couche et
# sans fonction d'activation est bien linéaire et a donc les memes resultats que la regression lineaire

4.195047326616335


\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [45]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

In [18]:
# Nb de paramètres dans self.Down1: (calcul "à la main")
# Down1 : 64 -> 128
# MaxPool1D(size = 2) 0
# Double_conv_casual avec in_ch = 64 et out_ch = 128
# batchnorm1d 2*n et relu 0
# Conv1 = 64*3*128 + 128
# Conv2 = 128*3*128 + 128
# Total Down1
# (64*3*128 + 128) + 2*128 + (128*3*128 + 128) + 2*128 = 74 496

# Nb de paramètres au total:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Nombre total de paramètres: {total_params}")

Nombre total de paramètres: 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

In [ ]:
# La taille est reduite avec les maxpooling de down causal
# Elle est restituée avec ConvTranspose et cat dans Up_causal

**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

In [ ]:
# Le champ réceptif est augmenté par des convolutions causales empilées.
#(kernel_size - 1) * nb_convolutions + 1 = (3 - 1) * 2 + 1 = 5.
# self.inc a un champ réceptif de 5

**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

In [46]:
input_tensor2 = input_tensor1.clone()
input_tensor2[0, 0, 0] = -input_tensor1[0, 0, 0]

input_tensor2 = input_tensor1.clone()
input_tensor2[0, 0, 0] = 1  # Modifier une composante

print(input_tensor1[0,0,0])
print(input_tensor2[0,0,0])

# Obtenir les sorties du modèle pour les deux entrées
output1 = model(input_tensor1)
output2 = model(input_tensor2)

# Trouver la taille du champ réceptif
receptive_field_size = -1
for i in range(len(output1[0, 0])):
    if output1[0, 0, i] == output2[0, 0, i]:
      receptive_field_size = i
      break

print("Taille du champ réceptif :", receptive_field_size)


tensor(0.2244)
tensor(1.)
Taille du champ réceptif : 5916


**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?